<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3c3ab6f466ef8b92284e6165a7ca624c54f515d8ff4812061858615704f6b3c4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-04 09:14:08
-------------------
qualified stocks: 90
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.00 L
Current:  1.40 C
-------------------
Today PnL: 0.00 (0.0%)
Current PnL: -17.74 L (-11.93%)
CY Booked + Current PnL: -8.18 L (-5.5%)
-------------------
Total profit:  3.12 L
Total loss:  -20.86 L
-------------------
Total Booked + Current PnL: 18.80 L (15.45%)
Total Booked PnL: 36.55 L (30.03%)
Curr Year Booked PnL: 9.57 L (6.83%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.78 L (56.26%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-28.97,69.0,H-MC,7.13,142072.0,27136.0,9448.0,0.0,23.61,6.65,31.83,81.0,2.87,1.05,47.26,X40N,ATH,IT
80,TTKPRESTIG,770.00,107.26,71.0,M-SC,6.65,89322.0,-11455.0,11549.0,-0.0,-11.37,12.93,0.09,245.0,-0.99,0.66,17.35,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.86,58.0,H-LC,9.20,170192.0,14015.0,12084.0,0.0,8.97,7.10,16.71,22.0,1.16,1.26,27.98,X40N,BTT,HEALTHCARE
65,SBIN,863.00,-13.25,51.0,M-LC,4.50,215220.0,13740.0,13473.0,0.0,6.82,6.26,13.51,60.0,1.02,1.59,20.28,XY25,NTT,BANKS
15,BRITANNIA,6362.67,17.62,65.0,H-LC,9.12,212688.0,14592.0,16377.0,0.0,7.37,7.70,15.63,26.0,0.89,1.58,30.82,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.14,62.0,M-MC,7.63,223309.0,-1653.0,169313.0,0.0,-0.73,75.82,74.53,192.0,-0.01,1.65,29.65,XY24,BTT,STEEL
48,KPIGREEN,731.35,4.80,44.0,H-SC,11.07,125093.0,-204.0,59207.0,0.0,-0.16,47.33,47.09,141.0,-0.00,0.93,56.56,MH,ATH,POWER
37,ICICIGI,2260.25,-21.50,33.0,H-MC,4.27,134828.0,-1305.0,32426.0,0.0,-0.96,24.05,22.86,91.0,-0.04,1.00,11.53,X40,ATH,INSURANCE
54,PGHH,18062.58,-30.02,56.0,H-MC,5.25,202425.0,1605.0,68521.0,0.0,0.80,33.85,34.92,80.0,0.02,1.50,5.51,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,0.65,42.0,H-LC,9.85,230178.0,-11746.0,75015.0,0.0,-4.86,32.59,26.15,8.0,-0.16,1.70,8.77,X40,ATH,IT
46,JIOFIN,387.00,-0.75,43.0,H-LC,13.98,219590.0,2184.0,51318.0,0.0,1.00,23.37,24.61,48.0,0.04,1.63,56.37,XY24,BTT,FINANCE
59,RELIANCE,1533.00,-14.41,43.0,H-LC,7.52,214126.0,3760.0,25031.0,0.0,1.79,11.69,13.68,37.0,0.15,1.59,18.61,XY25,NTT,REFINERIES
42,INFY,2275.00,-19.24,46.0,H-LC,11.05,315048.0,2160.0,169527.0,-0.0,0.69,53.81,54.87,3.0,0.01,2.33,7.39,X40,BTT,IT
77,TCS,4998.00,-27.08,48.0,H-LC,14.51,291165.0,-54791.0,178659.0,0.0,-15.84,61.36,35.80,1.0,-0.31,2.16,3.15,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-5.78,52.0,H-LC,2.14,233595.0,-12556.0,91756.0,0.0,-5.10,39.28,32.18,16.0,-0.14,1.73,29.10,X200,ATH,IT
53,NESTLEIND,1377.00,-9.11,60.0,H-LC,2.79,279583.0,14157.0,42636.0,-0.0,5.33,15.25,21.40,11.0,0.33,2.07,13.06,XY25,NTT,FMCG
44,ITC,452.00,-37.50,54.0,H-LC,3.03,201346.0,1208.0,19692.0,0.0,0.60,9.78,10.44,4.0,0.06,1.49,6.20,X40,NTT,FMCG
20,CIPLA,1795.00,-19.09,58.0,H-LC,5.39,216241.0,11741.0,29668.0,0.0,5.74,13.72,20.25,10.0,0.40,1.60,14.98,X40N,BTT,PHARMA
86,VBL,671.64,-10.56,53.0,H-LC,7.11,322350.0,6508.0,106150.0,0.0,2.06,32.93,35.67,5.0,0.06,2.39,16.20,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,122.36,41.0,H-SC,9.05,120004.0,-66928.0,76275.0,-0.0,-35.80,63.56,5.00,153.0,-0.88,0.89,14.80,OX40N,NTT,FINANCE
1,ABB,7934.00,-40.45,49.0,H-LC,13.23,249043.0,-12576.0,131794.0,-0.0,-4.81,52.92,45.57,7.0,-0.10,1.84,5.97,AR,NTT,ELECTRICAL
2,ACC,3906.00,-38.07,53.0,M-MC,6.68,134546.0,-53725.0,150597.0,-0.0,-28.54,111.93,51.45,174.0,-0.36,1.00,3.39,XY24,BTT,CEMENT
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,15.92,90678.0,-10285.0,104815.0,-0.0,-10.19,115.59,93.63,148.0,-0.10,0.67,35.06,SR,ATH,CHEMICALS
4,APOLLOHOSP,8285.00,-14.86,58.0,H-LC,9.20,170192.0,14015.0,12084.0,0.0,8.97,7.10,16.71,22.0,1.16,1.26,27.98,X40N,BTT,HEALTHCARE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,122.36,41.0,H-SC,9.05,120004.0,-66928.0,76275.0,-0.0,-35.80,63.56,5.00,153.0,-0.88,0.89,14.80,OX40N,NTT,FINANCE
1,ABB,7934.00,-40.45,49.0,H-LC,13.23,249043.0,-12576.0,131794.0,-0.0,-4.81,52.92,45.57,7.0,-0.10,1.84,5.97,AR,NTT,ELECTRICAL
2,ACC,3906.00,-38.07,53.0,M-MC,6.68,134546.0,-53725.0,150597.0,-0.0,-28.54,111.93,51.45,174.0,-0.36,1.00,3.39,XY24,BTT,CEMENT
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,15.92,90678.0,-10285.0,104815.0,-0.0,-10.19,115.59,93.63,148.0,-0.10,0.67,35.06,SR,ATH,CHEMICALS
4,APOLLOHOSP,8285.00,-14.86,58.0,H-LC,9.20,170192.0,14015.0,12084.0,0.0,8.97,7.10,16.71,22.0,1.16,1.26,27.98,X40N,BTT,HEALTHCARE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TTKPRESTIG,770.0,107.26,71.0,M-SC,6.65,89322.0,-11455.0,11549.0,-0.0,-11.37,12.93,0.09,245.0,-0.99,0.66,17.35,OX40N,NTT,DURABLES
47,KANSAINER,340.0,-68.16,56.0,H-SC,3.63,220113.0,-49554.0,85888.0,0.0,-18.38,39.02,13.47,138.0,-0.58,1.63,11.33,XY24,NTT,PAINTS
81,UJJIVANSFB,60.0,84.36,44.0,H-SC,13.50,116235.0,-26244.0,45762.0,0.0,-18.42,39.37,13.70,163.0,-0.57,0.86,37.19,OX40N,NTT,BANKS
40,INDIGOPNTS,1408.0,115.96,52.0,M-SC,3.31,141720.0,-32839.0,32879.0,0.0,-18.81,23.20,0.02,221.0,-1.00,1.05,23.27,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.53,43.0,H-SC,2.93,113544.0,-30259.0,57010.0,0.0,-21.04,50.21,18.60,149.0,-0.53,0.84,25.30,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-27.08,48.0,H-LC,14.51,291165.0,-54791.0,178659.0,0.0,-15.84,61.36,35.80,1.0,-0.31,2.16,3.15,X40,BTT,IT
42,INFY,2275.00,-19.24,46.0,H-LC,11.05,315048.0,2160.0,169527.0,-0.0,0.69,53.81,54.87,3.0,0.01,2.33,7.39,X40,BTT,IT
44,ITC,452.00,-37.50,54.0,H-LC,3.03,201346.0,1208.0,19692.0,0.0,0.60,9.78,10.44,4.0,0.06,1.49,6.20,X40,NTT,FMCG
86,VBL,671.64,-10.56,53.0,H-LC,7.11,322350.0,6508.0,106150.0,0.0,2.06,32.93,35.67,5.0,0.06,2.39,16.20,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.23,249043.0,-12576.0,131794.0,-0.0,-4.81,52.92,45.57,7.0,-0.10,1.84,5.97,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187215.0,36302.0,21006.0,0.0,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
26,DIXON,18940.15,-28.97,69.0,H-MC,7.13,142072.0,27136.0,9448.0,0.0,23.61,6.65,31.83,81.0,2.87,1.05,47.26,X40N,ATH,IT
27,DMART,5201.00,-6.62,70.0,H-LC,14.89,233166.0,48685.0,21684.0,-0.0,26.39,9.30,38.14,38.0,2.25,1.73,39.79,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,13.61,66.0,H-MC,3.40,196060.0,31390.0,36585.0,0.0,19.06,18.66,41.28,89.0,0.86,1.45,28.57,X40N,ATH,AC
87,VOLTAS,1918.49,2.28,67.0,H-MC,4.49,214545.0,22803.0,73224.0,0.0,11.89,34.13,50.08,99.0,0.31,1.59,19.50,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187215.0,36302.0,21006.0,0.0,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
24,DABUR,735.00,-2.58,64.0,H-MC,4.24,211965.0,14937.0,74675.0,0.0,7.58,35.23,45.49,102.0,0.20,1.57,21.61,XY24,BTT,FMCG
18,CAMS,5211.76,-6.27,47.0,H-SC,5.53,106501.0,4497.0,39427.0,0.0,4.41,37.02,43.06,122.0,0.11,0.79,22.28,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187215.0,36302.0,21006.0,0.0,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
52,NATIONALUM,247.44,-45.32,71.0,H-MC,0.24,109774.0,9270.0,21373.0,0.0,9.22,19.47,30.49,79.0,0.43,0.81,44.62,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-40.29,58.0,M-SC,4.68,100868.0,9370.0,69387.0,0.0,10.24,68.79,86.07,223.0,0.14,0.75,46.48,XR,NTT,DURABLES
39,INDIAMART,4850.92,-52.03,43.0,H-SC,12.47,133973.0,10637.0,123121.0,-0.0,8.62,91.90,108.45,119.0,0.09,0.99,32.91,AR,ATH,MISC
89,WIPRO,420.00,-11.98,50.0,M-LC,6.90,154727.0,3782.0,105679.0,-0.0,2.51,68.30,72.51,53.0,0.04,1.15,8.68,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SIS,528.00,2251.81,56.0,H-SC,5.75,63274.0,-17816.0,26486.0,0.0,-21.97,41.86,10.69,156.0,-0.67,0.47,28.23,OX40N,NTT,MISC
63,SAMMAANCAP,326.00,-172.73,68.0,M-SC,6.69,83166.0,-19044.0,112432.0,-0.0,-18.63,135.19,91.37,208.0,-0.17,0.62,35.01,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,15.92,90678.0,-10285.0,104815.0,-0.0,-10.19,115.59,93.63,148.0,-0.10,0.67,35.06,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,35.47,92179.0,-21370.0,61419.0,-0.0,-18.82,66.63,35.27,268.0,-0.35,0.68,125.61,XR,NTT,HOTELS
51,MASFIN,398.61,-17.34,54.0,H-SC,13.28,93750.0,-4230.0,25838.0,0.0,-4.32,27.56,22.05,152.0,-0.16,0.69,36.27,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,35.47,92179.0,-21370.0,61419.0,-0.0,-18.82,66.63,35.27,268.0,-0.35,0.68,125.61,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,20.36,187215.0,36302.0,21006.0,0.0,24.05,11.22,37.97,93.0,1.73,1.39,55.45,AR,ATH,AUTO
58,RELAXO,1176.00,-31.16,63.0,H-SC,5.46,63923.0,-51615.0,86603.0,-0.0,-44.67,135.48,30.28,136.0,-0.60,0.47,25.67,X40N,NTT,FOOTWEAR
27,DMART,5201.00,-6.62,70.0,H-LC,14.89,233166.0,48685.0,21684.0,-0.0,26.39,9.30,38.14,38.0,2.25,1.73,39.79,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,13.61,66.0,H-MC,3.40,196060.0,31390.0,36585.0,0.0,19.06,18.66,41.28,89.0,0.86,1.45,28.57,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,43.41
2,50,73.46


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.45
SC    37.17
MC    25.39
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.25
X40      15.38
X40N     14.25
XY25     11.49
AR        9.35
XR        8.96
OX40N     7.34
MH        1.74
X200      1.73
X5K       1.43
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.27
H-MC    22.42
H-SC    22.27
M-SC    13.31
M-LC     7.11
M-MC     2.65
L-SC     1.59
L-LC     1.07
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.34,1.34,30.80
IT,13.27,-15.25,72.40
FINANCE,7.32,-22.10,67.11
BANKS,7.28,-15.79,64.96
PAINTS,5.97,-14.50,40.12
MISC,5.81,-17.24,84.45
ELECTRICAL,5.66,-7.74,67.12
AUTO,4.59,-9.06,56.51
AC,3.76,4.77,27.49


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2788709.0,22
AR,1216741.0,10
XR,1082621.0,13
X40,889889.0,10
X40N,509296.0,11
OX40N,466655.0,10
XY25,431250.0,8
SR,196324.0,2
X5K,124241.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2770002.0,27
M-SC,1449527.0,17
H-LC,1397739.0,17
H-MC,1177324.0,17
M-LC,418047.0,5
M-MC,319910.0,2
L-SC,246673.0,3
L-MC,58817.0,1
L-LC,40023.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,926821.0,6
M-SC,XY24,862101.0,7
H-SC,AR,666683.0,5
H-LC,X40,592145.0,5
H-SC,XR,524601.0,6
H-MC,XY24,453905.0,4
H-LC,AR,369699.0,2
M-MC,XY24,319910.0,2
H-MC,X40,220252.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
